<h2 style='color:purple' align='center'>Naive Bayes Tutorial Part 1: Predicting survival from titanic crash</h2>

In [21]:
import pandas as pd

In [22]:
df = pd.read_csv("/content/titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [23]:
df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)
df.head()

,Survived,Pclass,Sex,Age,Fare
0,0,3,male,22.0,7.2500
1,1,1,female,38.0,71.2833
2,1,3,female,26.0,7.9250
3,1,1,female,35.0,53.1000
4,0,3,male,35.0,8.0500


In [24]:
inputs = df.drop('Survived',axis='columns')
target = df.Survived

In [25]:
#inputs.Sex = inputs.Sex.map({'male': 1, 'female': 2})

In [26]:
dummies = pd.get_dummies(inputs.Sex)
dummies.head(3)

,female,male
0,False,True
1,True,False
2,True,False


In [27]:
inputs = pd.concat([inputs,dummies],axis='columns')
inputs.head(3)

,Pclass,Sex,Age,Fare,female,male
0,3,male,22.0,7.2500,False,True
1,1,female,38.0,71.2833,True,False
2,3,female,26.0,7.9250,True,False


**I am dropping male column as well because of dummy variable trap theory. One column is enough to repressent male vs female**

In [28]:
inputs.drop(['Sex','male'],axis='columns',inplace=True)
inputs.head(3)

,Pclass,Age,Fare,female
0,3,22.0,7.2500,False
1,1,38.0,71.2833,True
2,3,26.0,7.9250,True


In [29]:
inputs.columns[inputs.isna().any()]

Index(['Age'], dtype='object')

In [30]:
inputs.Age[:10]

,Age
0,22.0
1,38.0
2,26.0
3,35.0
4,35.0
5,NaN
6,54.0
7,2.0
8,27.0
9,14.0


In [31]:
inputs.Age = inputs.Age.fillna(inputs.Age.mean())
inputs.head()

,Pclass,Age,Fare,female
0,3,22.0,7.2500,False
1,1,38.0,71.2833,True
2,3,26.0,7.9250,True
3,1,35.0,53.1000,True
4,3,35.0,8.0500,False


In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs,target,test_size=0.3)

In [33]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [34]:
model.fit(X_train,y_train)

GaussianNB()

In [35]:
model.score(X_test,y_test)

0.7649253731343284

In [36]:
X_test[0:10]

,Pclass,Age,Fare,female
276,3,45.000000,7.7500,True
842,1,30.000000,31.0000,True
566,3,19.000000,7.8958,False
387,2,36.000000,13.0000,True
382,3,32.000000,7.9250,False
253,3,30.000000,16.1000,False
748,1,19.000000,53.1000,False
291,1,19.000000,91.0792,True
306,1,29.699118,110.8833,True
623,3,21.000000,7.8542,False


In [37]:
y_test[0:10]

,Survived
276,0
842,1
566,0
387,1
382,0
253,0
748,0
291,1
306,1
623,0


In [38]:
model.predict(X_test[0:10])

array([1, 1, 0, 1, 0, 0, 0, 1, 1, 0])

In [39]:
model.predict_proba(X_test[:10])

array([[0.49138612, 0.50861388],
       [0.09352034, 0.90647966],
       [0.9640693 , 0.0359307 ],
       [0.31058888, 0.68941112],
       [0.97162258, 0.02837742],
       [0.97179675, 0.02820325],
       [0.61785422, 0.38214578],
       [0.00621004, 0.99378996],
       [0.00158089, 0.99841911],
       [0.96593899, 0.03406101]])

**Calculate the score using cross validation**

In [40]:
from sklearn.model_selection import cross_val_score
cross_val_score(GaussianNB(),X_train, y_train, cv=5)

array([0.8       , 0.784     , 0.728     , 0.75806452, 0.79032258])